In [1]:
import sys
import os

# Add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

sys.path.append(os.path.abspath("../src"))





In [3]:
import sys
import os

# Add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

sys.path.append(os.path.abspath("../src"))





In [4]:
# --- 1. Import necessary libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd
from src.yf_tesla_modeling import find_best_arima_params

In [5]:
data = pd.read_csv(r"C:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\data\combined_2015-07-01_to_2025-08-01.csv")
# --- 2. Prepare the DataFrame ---
# Convert the 'Date' column into a proper datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'Date' column as the index, which is standard for time series analysis
data.set_index('Date', inplace=True)

# --- 3. Select and Clean the TSLA Data (as we discussed) ---
# Now you can select the 'TSLA.Adj Close' column, which we know exists
tsla_df = data[['TSLA.Adj Close']]

# Rename the column to make the rest of the code simpler
tsla_df = tsla_df.rename(columns={'TSLA.Adj Close': 'Adj Close'})

# Clean the data to ensure business day frequency and fill gaps
tsla_df = tsla_df.asfreq('B').ffill().bfill()


# --- 4. Verify the result ---
print("Data loaded and prepared successfully!")
print("Here are the first 5 rows of your clean TSLA data:")
print(tsla_df.head())

Data loaded and prepared successfully!
Here are the first 5 rows of your clean TSLA data:
            Adj Close
Date                 
2015-07-01  17.943333
2015-07-02  18.667999
2015-07-03  18.667999
2015-07-06  18.648001
2015-07-07  17.858667


In [6]:
split_date = '2024-01-01'
train = tsla_df[tsla_df.index < split_date]
test = tsla_df[tsla_df.index >= split_date]
# Cell 3: Call the function and get the best order
# Pass the correct Series to your function
# Corrected code with .dropna()
auto_model = find_best_arima_params(train['Adj Close'])

# .order is an attribute (a variable), not a function. You access it directly.
best_order = auto_model.order 

print(f"\nBest ARIMA order found: {best_order}")
# --- 2. LSTM Data Preparation ---
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train)
scaled_test_data = scaler.transform(test)

Running auto_arima to find the best model parameters...
Performing stepwise search to minimize aic


c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ABC\Desktop\10Acadamy\w

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=14066.624, Time=1.13 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=14065.901, Time=0.13 sec


c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=14065.756, Time=0.18 sec


c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=14065.762, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=14064.621, Time=0.10 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.772 seconds

Best ARIMA order found: (0, 1, 0)


c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf

# --- ASSUMPTIONS ---
# This script assumes you have the following variables from your previous steps:
# - scaler: The fitted MinMaxScaler object.
# - tsla_df: The full, cleaned DataFrame of historical TSLA prices.

# --- 1. Define the time_step ---
# This must be an integer. We will use 60 as it's a common standard.
time_step = 60

# --- 2. Load the Trained Model ---
model_filepath = r'C:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\models\tsla_lstm_model.keras'
trained_model = tf.keras.models.load_model(model_filepath)
print("Trained LSTM model loaded successfully.")

# --- 3. Prepare the Initial Input for Forecasting ---
# We need the last 'time_step' days from our historical data to start the forecast.
last_sequence = tsla_df['Adj Close'][-time_step:].values.reshape(-1, 1)

# Scale this initial input
scaled_last_sequence = scaler.transform(last_sequence)

# --- 4. Iterative Forecasting Loop ---
future_predictions = []
current_input = scaled_last_sequence.reshape(1, time_step, 1)
forecast_days = 252  # Approximately 1 year of trading days

print(f"\nGenerating forecast for the next {forecast_days} trading days...")

for _ in range(forecast_days):
    # Predict the next day
    next_prediction_scaled = trained_model.predict(current_input, verbose=0)
    
    # Store the scaled prediction
    future_predictions.append(next_prediction_scaled[0, 0])
    
    # Update the input sequence for the next prediction
   # Corrected code
# Reshape the 2D prediction to 3D before appending
current_input = np.append(current_input[:, 1:, :], next_prediction_scaled.reshape(1, 1, 1), axis=1)

# --- 5. Finalize the Forecast ---
# Inverse transform all predictions at once
future_forecast_values = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Create a future date index for the forecast
last_date = tsla_df.index[-1]
future_date_index = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=forecast_days)

# Create the final forecast DataFrame
future_forecast_df = pd.DataFrame(future_forecast_values, index=future_date_index, columns=['Forecast'])

print("Forecast generation complete.")
print("\n--- First 5 Days of Future Forecast ---")
print(future_forecast_df.head())

Trained LSTM model loaded successfully.

Generating forecast for the next 252 trading days...


c:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Forecast generation complete.

--- First 5 Days of Future Forecast ---
              Forecast
2025-08-01  291.193817
2025-08-04  291.193817
2025-08-05  291.193817
2025-08-06  291.193817
2025-08-07  291.193817


In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from tqdm import tqdm

# --- 1. Load Model and Prepare Data ---
# This assumes 'scaler' and 'tsla_df' are already loaded from your previous work.
# And that 'time_step' is defined (e.g., 60).
# --- 1. Define the time_step ---
# This must be an integer. We will use 60 as it's a common standard.
time_step = 10

model_filepath = r'C:\Users\ABC\Desktop\10Acadamy\week_11\Time-Series-Forecasting\models\tsla_lstm_model.keras'
# To use Monte Carlo Dropout, we need to reload the model to access the underlying layers
# --- Load the Model ---
base_model = tf.keras.models.load_model(model_filepath)

# --- Add this line to FIX the error ---
# This tells the model its expected input shape.
# (None = flexible batch size, time_step = your look-back, 1 = number of features)
# Corrected code
base_model.build(input_shape=(None, time_step, 1))

print("Model loaded and built successfully.")

# --- The rest of your code will now work ---
# Create a new model that runs the base model in training mode (dropout active)
inputs = tf.keras.Input(shape=(base_model.input.shape[1], base_model.input.shape[2]))
outputs = base_model(inputs, training=True)
mc_model = tf.keras.Model(inputs, outputs)

# Create a new model that runs the base model in training mode (dropout active)


print("Trained LSTM model loaded for Monte Carlo forecasting.")

# --- 2. Monte Carlo Iterative Forecasting ---
# We will run the forecast 50 times to get a distribution of possible outcomes.
n_iterations = 50
forecast_days = 180 # Approx. 6 months
time_step = 10 # Ensure this matches your model's training

# Prepare the initial input sequence from the last known data
last_sequence = tsla_df['Adj Close'][-time_step:].values.reshape(-1, 1)
scaled_last_sequence = scaler.transform(last_sequence)

# Store all the different forecast paths
all_forecast_paths = []

print(f"\nRunning {n_iterations} Monte Carlo simulations for a {forecast_days}-day forecast...")

for _ in tqdm(range(n_iterations), desc="Forecasting Progress"):
    current_input = scaled_last_sequence.reshape(1, time_step, 1)
    future_predictions_single_run = []
    
    for _ in range(forecast_days):
        next_prediction_scaled = mc_model.predict(current_input, verbose=0)
        future_predictions_single_run.append(next_prediction_scaled[0, 0])
        current_input = np.append(current_input[:, 1:, :], next_prediction_scaled.reshape(1, 1, 1), axis=1)
        
    all_forecast_paths.append(future_predictions_single_run)

# --- 3. Process the Forecasts to Get Mean and Confidence Intervals ---
# Convert paths to a NumPy array for easier calculation
all_forecast_paths = np.array(all_forecast_paths)

# Inverse transform all paths at once to get real dollar values
all_forecast_paths_unscaled = scaler.inverse_transform(all_forecast_paths.T).T

# Calculate the mean forecast and the confidence interval bounds (e.g., 95% interval)
mean_forecast = np.mean(all_forecast_paths_unscaled, axis=0)
lower_bound = np.percentile(all_forecast_paths_unscaled, 2.5, axis=0)
upper_bound = np.percentile(all_forecast_paths_unscaled, 97.5, axis=0)

# Create a future date index
last_date = tsla_df.index[-1]
future_date_index = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=forecast_days)

# Create the final forecast DataFrame
future_forecast_df = pd.DataFrame({
    'Forecast': mean_forecast,
    'Lower_CI': lower_bound,
    'Upper_CI': upper_bound
}, index=future_date_index)

print("Forecast and confidence intervals generated successfully.")

# --- 4. Visualize the Forecast with Confidence Intervals ---
# We will plot the last year of historical data for context
historical_data_to_plot = tsla_df.loc[tsla_df.index > (tsla_df.index.max() - pd.DateOffset(years=1))]

fig = go.Figure()

# Add the historical data
fig.add_trace(go.Scatter(
    x=historical_data_to_plot.index, y=historical_data_to_plot['Adj Close'],
    mode='lines', name='Historical Price', line=dict(color='royalblue')
))

# Add the lower and upper confidence interval bounds
fig.add_trace(go.Scatter(
    x=future_forecast_df.index, y=future_forecast_df['Upper_CI'],
    mode='lines', name='Upper 95% CI', line=dict(width=0)
))
fig.add_trace(go.Scatter(
    x=future_forecast_df.index, y=future_forecast_df['Lower_CI'],
    mode='lines', name='Lower 95% CI', line=dict(width=0),
    fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)' # Fill the area between upper and lower
))

# Add the mean forecast line
fig.add_trace(go.Scatter(
    x=future_forecast_df.index, y=future_forecast_df['Forecast'],
    mode='lines', name='Mean Forecast', line=dict(color='orange', width=3)
))

fig.update_layout(
    title='TSLA 6-Month Future Forecast with 95% Confidence Interval',
    xaxis_title='Date', yaxis_title='Adjusted Close Price',
    template='plotly_white', legend=dict(x=0.01, y=0.99)
)

fig.show()

Model loaded and built successfully.


AttributeError: The layer sequential has never been called and thus has no defined input.